# Import

In [ ]:
import re

def import_from_txt(filename):
    if not filename:
        filename = 'input_data.txt'
    with open(filename, 'r') as f:
        lines = f.readlines()

        #first line is the width of the silicon plate
        W = int(lines[0])

        #second line is the number of rectangles
        n = int(lines[1])
        print(str(W) +" "+ str(n))
        rectangles = []
        
        #read each rectangle
        for line in lines[2:2+n]:
            match = re.search(r"(\d+) (\d+)", line).group(1,2)
            x = int(match[0])
            y = int(match[1])
            rectangles.append((x,y))
        return W, n, rectangles

# Export

In [ ]:
import re

def export_to_txt(rect_vars, rectangles, H, W, n, filename = "output.txt" ):
    with open(filename[:-4]+"_output.txt", 'w') as f:
        f.write(str(W)+" "+f'{H.solution_value():.0f}'+ '\n')
        f.write(str(n)+'\n')
        for i in range(0, n):
            x,y,r = rect_vars[i]
            w, h = rectangles[i]
            f.write(
                str(w)+ " "+
                str(h)+ " "+
                f'{x.solution_value():.0f}'+ " "+
                f'{y.solution_value():.0f}'+ " "+
                f'{r.solution_value():.0f}'+ '\n'
            )

# Visualization

In [ ]:
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import math

from PIL import Image, ImageDraw
from cv2 import imshow
import matplotlib
import matplotlib.colors

S = 10

def plot(filename = "input_data.txt"):
    with open(filename[:-4] + "_output.txt", 'r') as f:
        lines = f.readlines()
        W, H = [int(value) for value in re.search(r"(\d+) (\d+)", lines[0]).groups()]

        grid = Image.new("RGBA", (W*S, (int(H*1.2))*S))
        grid1 = ImageDraw.Draw(grid)
        draw_grid(grid, grid1, S)
        #colors
        #green

        img = Image.new("RGBA", (W*S, (int(H*1.2))*S))
        img1 = ImageDraw.Draw(img)


        n = int(lines[1])
        for line in lines[2:]:
            w, h, x, y, r = [int(value) for value in re.search(r"(\d+) (\d+) (\d+) (\d+) (\d+)", line).groups()]
            print((w, h, x, y, r))
            img1.rectangle([(x*S,y*S),(S*(x+w*(1-r)+h*r),S*(y+h*(1-r)+w*r))], fill = (255,255,0,int(0.5*255)), outline='green', width = int(5*math.log(S,100)) )
        
        img = Image.alpha_composite(grid, img)
        img.save(filename[:-4] + ".png" )
        img.show()


def draw_grid(image, draw, S):
    # Draw some lines
    y_start = 0
    color = 'white'
    y_end = image.height
    step_size = 1*S

    for x in range(0, image.width, step_size):
        line = ((x, y_start), (x, y_end))
        draw.line(line, fill=color)

    x_start = 0
    x_end = image.width

    for y in range(0, image.height, step_size):
        line = ((x_start, y), (x_end, y))
        draw.line(line, fill=color)

# MIP Model

In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --upgrade --user ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
#from ortools.model_builder.python import model_builder_helper
import sys
import math

def main(filename, rotation = False, show_result = False):
    # Create the linear solver with the GLOP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return
    solver.EnableOutput()


    solver.set_time_limit(600000)

    W, n, rectangles = import_from_txt(filename) 

    # Create the constants
    X_LIMIT = W
    if rotation:
        Y_LIMIT = sum([max(rect[0], rect[1]) for rect in rectangles])
    else:
        Y_LIMIT = sum([rect[1] for rect in rectangles])
    M = 10000 #number to be high enough for the big M method
    #min_h=sum([rect[1]*rect[0] for rect in rectangles])//W
    # Create the variables
    rect_vars = []
    overlap_vars = []
    total_area = sum([w*h for (w,h) in rectangles])
    for i in range(0, n):
        if rotation:
            X_LIMIT = W - min(rectangles[i])
        else:
            X_LIMIT = W - rectangles[i][0]
        rect_vars.append((
            solver.IntVar(0, X_LIMIT, "x_"+str(i)),
            solver.IntVar(0, Y_LIMIT, "y_"+str(i)),
            solver.BoolVar( "r_"+str(i))
        ))
        overlap_vars.append([
            (
            #solver.IntVar(0, 1, "z1_"+str(i)+"_"+str(j)),
            #solver.IntVar(0, 1, "z2_"+str(i)+"_"+str(j)),
            #solver.IntVar(0, 1, "z3_"+str(i)+"_"+str(j)),
            #solver.IntVar(0, 1, "z4_"+str(i)+"_"+str(j))
            solver.BoolVar("z1_"+str(i)+"_"+str(j)),
            solver.BoolVar("z2_"+str(i)+"_"+str(j)),
            solver.BoolVar("z3_"+str(i)+"_"+str(j)),
            solver.BoolVar("z4_"+str(i)+"_"+str(j))
            ) for j in range(i+1, n)
        ])
    H = solver.IntVar(math.ceil(total_area/W), 2*math.ceil(total_area/W), "H")
 

    print('Number of variables =', solver.NumVariables())

    # Create a linear constraints
    visited_rect = []
    visited_indexes = []
    simmetry_breakings = 0
    # for each rectangle the fit is x + w(1-r) + h*r < W 
    #                           and y + h(1-r) + w*r < H
    for i in range(0, n):
        x, y, r = rect_vars[i]
        w, h = rectangles[i]
        solver.Add(x + w*(1-r) + h*r <= W)
        solver.Add(y + h*(1-r) + w*r <= H)   

        #similar rectangles simmetry breaking
        rect_dims = (h, w)
        if h < w or not rotation:
            rect_dims = (w, h)
        try:
            index = visited_rect.index(rect_dims)
            k = visited_indexes[index]
            x2,y2,r2 = rect_vars[k]
            z1, z2, z3, z4 = overlap_vars[k][i-k-1]
            solver.Add(z1 == 1)
            solver.Add(z4 == 1)
            print("aggiungo una rottura di simmetria tra i rettangoli %d e %d" % (i, k))
            simmetry_breakings = simmetry_breakings + 1
            visited_indexes[index] = i
        except ValueError as ve:
            visited_rect.append(rect_dims)
            visited_indexes.append(i)     

        for j in range(i+1, n):
            x2,y2,r2 = rect_vars[j]
            w2, h2 = rectangles[j]
            z1, z2, z3, z4 = overlap_vars[i][j-i-1]

            #z1 left, z2 right, z3 up, z4 down  || 0-active, 1-inactive

            solver.Add(x2 + w2*(1-r2) + h2*r2 <= x + M*z1)
            solver.Add(x + w*(1-r) + h*r <= x2 + M*z2)
            solver.Add(y + h*(1-r) + w*r <= y2 + M*z3)
            solver.Add(y2 + h2*(1-r2) + w2*r2 <= y + M*z4)
            solver.Add(z1 +z2 +z3 +z4 <= 3)

        #rotation simmetry breaking
        if (w==h) or not rotation: 
            solver.Add(r == 0)
            if rotation: simmetry_breakings = simmetry_breakings + 1
    #Lexicograph constraint of the two biggest rectangles
    rect_areas = [w*h for (w,h) in rectangles]
    max_index = rect_areas.index(max(rect_areas))
    rect_areas[max_index] = 0
    max_index_2 = rect_areas.index(max(rect_areas))
    if max_index > max_index_2:
        z1, z2, z3, z4 = overlap_vars[max_index_2][max_index - max_index_2 -1]
    else:
        z1, z2, z3, z4 = overlap_vars[max_index][max_index_2 - max_index -1]
    solver.Add(z1 == 1)
    solver.Add(z4 == 1)
    

    #size Gradient on the x axis simmetry breaking
    #the total constraint is sum(side[i]) < sum(1-side[i]) -----> 2*sum(side[i]) < n
    side_var = []
    grad_cons = solver.RowConstraint(n, 2*n, "grad_cons")
    for i in range(0, n):
        side_var.append(
            solver.BoolVar("side_"+str(i))
        )
        x, y, r = rect_vars[i]
        w, h = rectangles[i]
        side = side_var[i]
        solver.Add(x <= (W)/2  + W/2*side )
        solver.Add( x >= (W)/2 - W/2*(1-side) )
        grad_cons.SetCoefficient(side, 2)
            

    print('Number of constraints =', solver.NumConstraints())
    print('Number of simm breaking contraints =', simmetry_breakings)

    # Create the objective function.
    objective = solver.Objective()
    objective.SetCoefficient(H, 1)
    objective.SetMinimization()

    solver.SetSolverSpecificParametersAsString("conflict/lpiterations = 10")
    solver.SetSolverSpecificParametersAsString("display/verblevel = 4")
    solver.SetSolverSpecificParametersAsString("display/freq = 500")
    solver.SetSolverSpecificParametersAsString("branching/relpscost/filtercandssym = TRUE")
    solver.SetSolverSpecificParametersAsString("propagating/symmetry/onlybinarysymmetry = FALSE")
    solver.SetSolverSpecificParametersAsString("misc/usesymmetry = 7")
    
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value()) 
        print('Plate height = ', H.solution_value())
        export_to_txt(rect_vars, rectangles, H, W, n, filename)
    else:
        print('The problem does not have an optimal solution.')


    print('\nAdvanced usage:')
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
    if status == pywraplp.Solver.OPTIMAL and show_result:
            plot(filename)
    if status == pywraplp.Solver.OPTIMAL:
        return 1
    elif solver.wall_time() > 295000:
        return -1
    else: 
        return 0

# Run the model

In [ ]:
filename = "ins-12.txt"
main(filename)

19 14
Number of variables = 407
Number of constraints = 528
Number of simm breaking contraints = 0
Solution:
Objective value = 19.0
Plate height =  19.0

Advanced usage:
Problem solved in 15092.000000 milliseconds
Problem solved in 96492 iterations
Problem solved in 5388 branch-and-bound nodes


1

In [ ]:
def whole_run():
    prefix = "ins-"
    suffix = ".txt"
    i = 1
    fail_count = 0
    unfiseable = 0
    timeouts = 0
    while i<=40 and fail_count < 2:
        print("\nSolution of instance " + prefix + str(i) + suffix + ":")
        result = main(prefix + str(i) + suffix, True, False)
        if result < 0:
            unfiseable = unfiseable + 1
            fail_count += 1
        elif result < 1:
            timeouts += 1
            fail_count += 1
        i = i + 1

    print("-------------------------------------------------------------------------")
    print("Reached instance %d" % i)
    print("Timedout solutions: %d" % timeouts)
    print("Unfiseable solutions: %d" % unfiseable)

In [ ]:
whole_run()


Solution of instance ins-1.txt:
8 4
Number of variables = 37
aggiungo una rottura di simmetria tra i rettangoli 2 e 1
Number of constraints = 53
Number of simm breaking contraints = 3
Solution:
Objective value = 8.0
Plate height =  8.0

Advanced usage:
Problem solved in 31.000000 milliseconds
Problem solved in 28 iterations
Problem solved in 1 branch-and-bound nodes

Solution of instance ins-2.txt:
9 5
Number of variables = 56
Number of constraints = 74
Number of simm breaking contraints = 1
Solution:
Objective value = 12.0
Plate height =  12.0

Advanced usage:
Problem solved in 32.000000 milliseconds
Problem solved in 61 iterations
Problem solved in 1 branch-and-bound nodes

Solution of instance ins-3.txt:
10 6
Number of variables = 79
Number of constraints = 104
Number of simm breaking contraints = 2
Solution:
Objective value = 10.0
Plate height =  10.0

Advanced usage:
Problem solved in 58.000000 milliseconds
Problem solved in 66 iterations
Problem solved in 1 branch-and-bound node


Solution of instance ins-1.txt:
8 4
Number of variables = 37
aggiungo una rottura di simmetria tra i rettangoli 2 e 1
Number of constraints = 53
Number of simm breaking contraints = 3
Solution:
Objective value = 8.0
Plate height =  8.0

Advanced usage:
Problem solved in 20.000000 milliseconds
Problem solved in 2 iterations
Problem solved in 1 branch-and-bound nodes

Solution of instance ins-2.txt:
9 5
Number of variables = 56
Number of constraints = 74
Number of simm breaking contraints = 1
Solution:
Objective value = 12.0
Plate height =  12.0

Advanced usage:
Problem solved in 30.000000 milliseconds
Problem solved in 58 iterations
Problem solved in 1 branch-and-bound nodes

Solution of instance ins-3.txt:
10 6
Number of variables = 79
Number of constraints = 104
Number of simm breaking contraints = 2
Solution:
Objective value = 10.0
Plate height =  10.0

Advanced usage:
Problem solved in 43.000000 milliseconds
Problem solved in 66 iterations
Problem solved in 1 branch-and-bound nodes